In [1]:
import os

import numpy as np
from scipy.sparse import load_npz
from scipy.stats import pearsonr
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import pandas as pd

import torch
import torch_geometric
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch_geometric.loader import DataLoader
from torch_geometric.nn.conv import GATConv
from torch import sparse

import matplotlib.pyplot as plt
from torch_sparse import SparseTensor, matmul

In [2]:
adj_t = SparseTensor(
    row=torch.tensor([0, 0, 1, 0, 2, 3, 2]),  # Source nodes
    col=torch.tensor([1, 2, 2, 3, 1, 2, 0]),  # Target nodes
    value=torch.tensor([0.5, 0.8, 0.6, 0.1, 0.2, 0.9, 0.3]),  # Edge weights
    sparse_sizes=(4, 4)  # 3 nodes in total
)

In [3]:
adj_t

SparseTensor(row=tensor([0, 0, 0, 1, 2, 2, 3]),
             col=tensor([1, 2, 3, 2, 0, 1, 2]),
             val=tensor([0.5000, 0.8000, 0.1000, 0.6000, 0.3000, 0.2000, 0.9000]),
             size=(4, 4), nnz=7, density=43.75%)

In [4]:
adj_t.to_dense()

tensor([[0.0000, 0.5000, 0.8000, 0.1000],
        [0.0000, 0.0000, 0.6000, 0.0000],
        [0.3000, 0.2000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.9000, 0.0000]])

In [6]:
adj_t.set_value(None).to_dense()

tensor([[0., 1., 1., 1.],
        [0., 0., 1., 0.],
        [1., 1., 0., 0.],
        [0., 0., 1., 0.]])

In [8]:
edge_weights = torch.nn.Parameter(torch.rand(adj_t.nnz(), dtype=torch.float32))
#edge_weights = torch.tensor([1, 2, 3, 4], dtype=torch.float32)

In [9]:
adj_t_updated = adj_t.set_value(edge_weights)

/home/vdubey/.local/lib/python3.11/site-packages/torch_sparse/storage.py:15: UserWarning: `layout` argument unset, using default layout "coo". This may lead to unexpected behaviour.
  warnings.warn('`layout` argument unset, using default layout '


In [10]:
adj_t_updated.to_dense()

tensor([[0.0000, 0.7747, 0.7578, 0.1625],
        [0.0000, 0.0000, 0.4650, 0.0000],
        [0.5714, 0.5715, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3008, 0.0000]], grad_fn=<IndexPutBackward0>)

In [8]:
x = matmul(adj_t_updated, torch.tensor([[2, 3, 4, 5], [2, 3, 4, 5]]).view(4, 2))
x

tensor([[0, 0],
        [0, 0],
        [0, 0],
        [0, 0]])

In [9]:
torch.tanh(x)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])

In [10]:
edge_weights = F.dropout(edge_weights, p=0.9)

In [11]:
edge_weights

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 9.2030, 0.0000],
       grad_fn=<MulBackward0>)

In [12]:
x = matmul(adj_t, torch.tensor([[2, 3, 4, 5], [2, 3, 4, 5]]).view(4, 2))
x

tensor([[0, 0],
        [0, 0],
        [0, 0],
        [0, 0]])

In [13]:
adj_t

SparseTensor(row=tensor([0, 0, 0, 1, 2, 2, 3]),
             col=tensor([1, 2, 3, 2, 0, 1, 2]),
             val=tensor([0.5000, 0.8000, 0.1000, 0.6000, 0.3000, 0.2000, 0.9000]),
             size=(4, 4), nnz=7, density=43.75%)

In [14]:
adj_t = SparseTensor(
    row=torch.tensor([0, 1, 1, 2, 2]),  # Source nodes
    col=torch.tensor([2, 0, 2, 0, 1]),  # Target nodes
    sparse_sizes=(3, 3)  # 3 nodes in total
)

adj_t

SparseTensor(row=tensor([0, 1, 1, 2, 2]),
             col=tensor([2, 0, 2, 0, 1]),
             size=(3, 3), nnz=5, density=55.56%)

In [18]:
x = torch.randn(9).view(3, 3)
x

tensor([[-0.6833, -1.1793,  1.2907],
        [-0.8630,  0.6198,  1.3872],
        [-0.8458, -0.3886,  0.6923]])

In [21]:
m = nn.Linear(3, 1)
m(matmul(adj_t, x))

tensor([[-1.2857],
        [-2.2096],
        [-2.4411]], grad_fn=<AddmmBackward0>)